In [ ]:
import cv2
import mahotas
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn.decomposition import PCA

In [ ]:
# Source: https://github.com/credo-science/CREDO-ML/blob/main/offline_trigger/Wavelet_BINARY_ALFA.ipynb
def loadData(path, verbose=True):

  import cv2
  import glob

  dots = []
  lines = []
  worms = []
  artefacts = []

  for img in glob.glob(path+"hits_votes_4_Dots/*.png"):
      n = cv2.imread(img)
      dots.append(n)
  target_dots = [0 for _ in dots]

  for img in glob.glob(path+"hits_votes_4_Lines/*.png"):
      n = cv2.imread(img)
      lines.append(n)
  target_lines = [1 for _ in lines]

  for img in glob.glob(path+"hits_votes_4_Worms/*.png"):
      n = cv2.imread(img)
      worms.append(n)
  target_worms = [2 for _ in worms]

#   for img in glob.glob(path+"artefacts/*.png"):
#       n = cv2.imread(img)
#       artefacts.append(n)
#   target_artefacts = [3 for _ in artefacts]

  images=dots+lines+worms#+artefacts

  #target_signals_binary = [0 for _ in (dots+lines+worms)]
  #target_artefacts_binary = [1 for _ in artefacts]

  #targets=target_signals_binary+target_artefacts_binary
  targets=target_dots+target_lines+target_worms#+target_artefacts

  if verbose:
    print(len(images),len(targets))
    print(images[0].shape)
    print(len(dots), len(lines), len(worms), len(artefacts))


  return (images, targets)

In [ ]:
def pca_reduction(X: np.ndarray):
    """ Function for Principal Component Analysis (PCA)

    Args:
        X (np.ndarray): Input data.

    Returns:
        ret (np.ndarray): Data after PCA.
        ratio (np.float64): sum of explained_variance_ratio_
    """
    X_norm = preprocessing.normalize(X)

    pca = PCA(n_components=2)
    pca.fit(X_norm)

    ret=pca.transform(X_norm)

    ratio = pca.explained_variance_ratio_.sum()

    return ret, ratio

In [ ]:
def preprocess_data(X: list, dim: int, blur_ksize: tuple, thresh_min: int, thresh_max: int, zernike_radius: int, zernike_degree: int) -> np.ndarray:
    """Preprocess data.

    Args:
        X (list): Input data.
        dim (int): PCA dimensionality.
        blur_ksize (tuple): Blur kernel size.
        thresh_min (int): Threshold minimal value.
        thresh_max (int): Threshold maximal value.
        zernike_radius (int): Zernike moments radius.
        zernike_degree (int): Zernike moments degree.

    Returns:
        ret (np.ndarray): Preprocessed data.

    """
    # TODO: remove below code after pca is implemented
    # get number of zernike moments for given degree
    dim = mahotas.features.zernike_moments(
        cv2.cvtColor(X[0], cv2.COLOR_RGB2GRAY), zernike_radius, zernike_degree
        ).shape[0]
    
    ret: np.ndarray = np.zeros((len(X), dim))
    for i in range(len(X)):

        # Preprocess image before extracting zernike moments
        img: np.ndarray = cv2.cvtColor(X[i], cv2.COLOR_RGB2GRAY)
        img = cv2.GaussianBlur(img, blur_ksize, 0)
        _, img = cv2.threshold(img, thresh_min, thresh_max, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        
        # Extract zernike moments
        zernike: np.ndarray = mahotas.features.zernike_moments(img, zernike_radius, zernike_degree)
        
        ret[i,:] = zernike
    
    ret, _ = pca_reduction(ret)
        
    return ret

In [ ]:
path="../data/hit-images-final/"

X, y = loadData(path)

In [ ]:
dim = 0
blur_ksize = (5, 5)
thresh_min = 0
thresh_max = 255 
zernike_radius = 30 
zernike_degree = 8

X_= preprocess_data(X, dim, blur_ksize, thresh_min, thresh_max, zernike_radius, zernike_degree)
